In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider
from IPython.display import display
import glob
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [8]:
# PCA over all the frames - flattened

# importing all the data, getting all the frames

TRAINING_DATA_PATH = "../data/training_data/"
files = sorted(glob.glob(TRAINING_DATA_PATH + "mixed_samples/*.npy"))

heart_samples = []

for f in files:
    data = np.load(f)
    heart_samples.append(data)
heart_samples = np.array(heart_samples)
 
print(heart_samples.shape)

heart_xyz = heart_samples[..., :3]
X = heart_xyz.reshape(len(heart_xyz), -1)
print(X.shape)

pca = PCA(n_components = 50)
X_pca = pca.fit_transform(X)
print(pca.explained_variance_ratio_)
print(X_pca.shape)

(900, 10, 18000, 4)
(900, 540000)
[0.30516536 0.2456048  0.09653941 0.04462171 0.02640237 0.01987551
 0.01842039 0.01475463 0.01414272 0.01354737 0.0124781  0.01121955
 0.01086073 0.00971639 0.00848699 0.00800505 0.00715335 0.00680103
 0.00578201 0.00484285 0.00412508 0.00342057 0.00253415 0.00245203
 0.00238893 0.00235308 0.00233916 0.00232312 0.00231017 0.00229456
 0.00225984 0.00222639 0.00221539 0.00210934 0.00210201 0.00203367
 0.00201258 0.00197145 0.00189151 0.00183745 0.00179943 0.00175981
 0.00171535 0.0016925  0.00167379 0.00162896 0.00161028 0.00158567
 0.00153135 0.00149566]
(900, 50)
